<a href="https://colab.research.google.com/github/JohamSMC/electiva4/blob/master/python/Taller_DataSet_HorseColic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
# Importarcion librerias
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Cargar Datos
dataSet = pd.read_csv('/content/drive/My Drive/9° Semestre/Colab Notebooks/DataSets/dataSet-horseColic.txt', sep=' ', na_values=['?'])
dataSet

In [ ]:
# Información DataSet
dataSet.info()

In [ ]:
# Eliminación de atributos
dataSet = dataSet.drop(['surgery'],axis=1)
dataSet = dataSet.drop(['Hospital_Number'],axis=1)
dataSet = dataSet.drop(['type_of_lesion_1'],axis=1)
dataSet = dataSet.drop(['type_of_lesion_2'],axis=1)
dataSet = dataSet.drop(['type_of_lesion_3'],axis=1)
dataSet = dataSet.drop(['cp_data'],axis=1)
dataSet.head()

In [ ]:
# Valores de la clase
dataSet.outcome.value_counts()

In [ ]:
# Estadisticas descriptivas de dataSet
statistics = dataSet.describe()
print(statistics)

In [ ]:
# Limpieza de datos perdidos en el atriburo "outcome" por el minimo valor
dataSet.outcome = dataSet.outcome.fillna(1.0)

# Limpieza de los demas datos por la media
for column in dataSet:
  missing_data = dataSet.loc[:,column].isna()
  dataSet.loc[missing_data,column]= statistics.loc['mean',column]

dataSet.head()

In [24]:
# Separa atributos de la clase("outcome")

# Cambio tipo de variablde de outcome de int a string
dataSet.outcome = dataSet.outcome.astype(str)

# Cambia todo los "outcome" de valor 3.0 a 2.0 ya que:
# 1 = Vivo
# 2 = Muerto
# 3 = Eeutanasia
dataSet.outcome = dataSet.outcome.replace({'3.0':2.0})

In [88]:
X = dataSet.drop(['outcome'],axis=1)
y = dataSet['outcome']

# **Splitting Data and SMOTE**

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 12345)

# Smote
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 12345) 
#X_train, y_train= sm.fit_sample(X_train, y_train) 

# **Árbol de decisión**

In [90]:
clf = DecisionTreeClassifier()
# Entrenar Decision Tree Classifer
clf = clf.fit(X_train,y_train)

# Predecir la respuesta para el conjunto de datos de prueba
y_pred = clf.predict(X_test)

In [91]:
# Resultados

#print("Numbero de observaciones mal etiquetadas del total de %d observaciones : %d" %(X_test.shape[0], (y_test != y_pred).sum()))
print("Numero de datos en test:",X_test.shape[0])
print("Numero de aciertos:",metrics.accuracy_score(y_test,y_pred,normalize=False))
print("Numero de desaciertos:",(y_test != y_pred).sum())
print("Exactitud(Accuracy):",metrics.accuracy_score(y_test,y_pred))

# Matriz de Confución
print("\n\t---Matriz de Confución Árbol de decisión---")
# print(metrics.confusion_matrix(y_test,y_pred))
VN,FP,FN,VP = metrics.confusion_matrix(y_test,y_pred).ravel()
print("VN \t FP")
print(VN," \t ",FP)
print("FN \t VP")
print(FN," \t ",VP)

Numero de datos en test: 90
Numero de aciertos: 61
Numero de desaciertos: 29
Exactitud(Accuracy): 0.6777777777777778

	---Matriz de Confución Árbol de decisión---
VN 	 FP
37  	  16
FN 	 VP
13  	  24


# **Gaussian Naive Bayes**

In [92]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [93]:
# Resultados

#print("Numbero de observaciones mal etiquetadas del total de %d observaciones : %d" %(X_test.shape[0], (y_test != y_pred).sum()))
print("Numero de datos en test:",X_test.shape[0])
print("Numero de aciertos:",metrics.accuracy_score(y_test,y_pred,normalize=False))
print("Numero de desaciertos:",(y_test != y_pred).sum())
print("Exactitud(Accuracy):",metrics.accuracy_score(y_test,y_pred))

# Matriz de Confución
print("\n\t---Matriz de Confución Naive Bayes---")
# print(metrics.confusion_matrix(y_test,y_pred))
VN,FP,FN,VP = metrics.confusion_matrix(y_test,y_pred).ravel()
print("VN \t FP")
print(VN," \t ",FP)
print("FN \t VP")
print(FN," \t ",VP)

Numero de datos en test: 90
Numero de aciertos: 63
Numero de desaciertos: 27
Exactitud(Accuracy): 0.7

	---Matriz de Confución Naive Bayes---
VN 	 FP
37  	  16
FN 	 VP
11  	  26


# ***OneR***

In [94]:
n_samples, n_features = X.shape
attribute_means = X.mean(axis=0)
assert attribute_means.shape == (n_features,)
X_d = np.array(X >= attribute_means, dtype='int')

In [95]:
from collections import defaultdict
from operator import itemgetter

def train(X, y_true, feature):  
    # Check that variable is a valid number
    n_samples, n_features = X.shape
    assert 0 <= feature < n_features
    # Get all of the unique values that this variable has
    values = set(X[:,feature])
    # Stores the predictors array that is returned
    predictors = dict()
    errors = []
    for current_value in values:
        most_frequent_class, error = train_feature_value(X, y_true, feature, current_value)
        predictors[current_value] = most_frequent_class
        errors.append(error)
    # Compute the total error of using this feature to classify on
    total_error = sum(errors)
    return predictors, total_error
# Compute what our predictors say each sample is based on its value
#y_predicted = np.array([predictors[sample[feature]] for sample in X])
def train_feature_value(X, y_true, feature, value):
    # The four input parameters are data set, category array, selected feature index value and feature value
    # Create a simple dictionary to count how frequency they give certain predictions
    class_counts = defaultdict(int)
    # Iterate through each sample and count the frequency of each class/value pair
    for sample, y in zip(X, y_true):
        if sample[feature] == value:
            class_counts[y] += 1
    # Now get the best one by sorting (highest first) and choosing the first item
    sorted_class_counts = sorted(class_counts.items(), key=itemgetter(1), reverse=True)
    most_frequent_class = sorted_class_counts[0][0]
    # The error is the number of samples that do not classify as the most frequent class
    # *and* have the feature value.
    n_samples = X.shape[1]
    error = sum([class_count for class_value, class_count in class_counts.items()
                 if class_value != most_frequent_class])
    return most_frequent_class, error

In [96]:
# Now, we split into a training and test set
X_train, X_test, y_train, y_test = train_test_split(X_d, y, test_size = 0.3, random_state = 12345)
# Smote
#X_train, y_train= sm.fit_sample(X_train, y_train) 

In [97]:
# Compute all of the predictors
all_predictors = {variable: train(X_train, y_train, variable) for variable in range(X_train.shape[1])}
errors = {variable: error for variable, (mapping, error) in all_predictors.items()}
# Now choose the best and save that as "model"
# Sort by error
best_variable, best_error = sorted(errors.items(), key=itemgetter(1))[0]
#print("The best model is based on variable {0} and has error {1:.2f}".format(best_variable, best_error))

# Choose the bset model
model = {'variable': best_variable,
         'predictor': all_predictors[best_variable][0]}

def predict(X_test, model):
    variable = model['variable']
    predictor = model['predictor']
    y_pred = np.array([predictor[int(sample[variable])] for sample in X_test])
    return y_pred

y_pred = predict(X_test, model)

In [98]:
# Resultados

#print("Numbero de observaciones mal etiquetadas del total de %d observaciones : %d" %(X_test.shape[0], (y_test != y_pred).sum()))
print("Numero de datos en test:",X_test.shape[0])
print("Numero de aciertos:",metrics.accuracy_score(y_test,y_pred,normalize=False))
print("Numero de desaciertos:",(y_test != y_pred).sum())
print("Exactitud(Accuracy):",metrics.accuracy_score(y_test,y_pred))
print("\n\t---Matriz de Confución OneR---")
# print(metrics.confusion_matrix(y_test,y_pred))
VN,FP,FN,VP = metrics.confusion_matrix(y_test,y_pred).ravel()
print("\nVP \t FN")
print(VP," \t ",FN)
print("FP \t VN")
print(FP," \t ",VN)

Numero de datos en test: 90
Numero de aciertos: 50
Numero de desaciertos: 40
Exactitud(Accuracy): 0.5555555555555556

	---Matriz de Confución OneR---

VP 	 FN
19  	  18
FP 	 VN
22  	  31
